In [115]:
import pandas as pd
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

### Relevant Data Set can be found on : https://data.world/raghav333/cricket-players-espn

## Reading Data

#### PySpark

In [116]:
spark=SparkSession.builder.appName('temp').getOrCreate()

#### Pandas

In [117]:
main=pd.read_csv("cricket_data.csv")

E:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,16,20,21,22,24,25,26,27,28,29,30,31,32,45,46,59,60,73,74,87,88,90,91,92,95,96,97,98,99,100,101,102) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Converting Pandas Dtype Object to String Dtype in order to use the same dateframe as Spark DataFrame

In [118]:
cols_dtypes=main.dtypes.values.tolist()
columns=main.columns.values.tolist()
for index,_ in enumerate(cols_dtypes):
    if str(cols_dtypes[index])=="object":
        main[columns[index]]=main[columns[index]].astype(str)

In [ ]:
main_spark=spark.createDataFrame(main)

In [ ]:
main_spark.printSchema()

root
 |-- ID: long (nullable = true)
 |-- NAME: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- Full name: string (nullable = true)
 |-- Born: string (nullable = true)
 |-- Died: string (nullable = true)
 |-- Current age: string (nullable = true)
 |-- Major teams: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Nickname: double (nullable = true)
 |-- Playing role: string (nullable = true)
 |-- Batting style: string (nullable = true)
 |-- Bowling style: string (nullable = true)
 |-- Other: string (nullable = true)
 |-- Relation: string (nullable = true)
 |-- In a nutshell: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- AWARDS: string (nullable = true)
 |-- BATTING_Tests_Mat: double (nullable = true)
 |-- BATTING_Tests_Inns: string (nullable = true)
 |-- BATTING_Tests_NO: string (nullable = true)
 |-- BATTING_Tests_Runs: string (nullable = true)
 |-- BATTING_Tests_HS: string (nulla

In [ ]:
main.head()

,ID,NAME,COUNTRY,Full name,Born,Died,Current age,Major teams,Education,Height,...,BOWLING_T20s_Runs,BOWLING_T20s_Wkts,BOWLING_T20s_BBI,BOWLING_T20s_BBM,BOWLING_T20s_Ave,BOWLING_T20s_Econ,BOWLING_T20s_SR,BOWLING_T20s_4w,BOWLING_T20s_5w,BOWLING_T20s_10
0,8772,Henry Arkell,England,Henry John Denham Arkell,"\nJune 26, 1898, Edmonton, Middlesex","March 12, 1982, Oxford (aged 83 years 259 days)",nan,Northamptonshire,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,532565,Richard Nyren,England,Richard Nyren,"\nApril 25, 1734, Eartham, Sussex","April 25, 1797, Lee or Leigh, Kent (aged 63 ye...",nan,Hampshire XI,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,16856,Sydney Maartensz,England,Sydney Gratien Adair Maartensz,"\nApril 14, 1882, Colombo, Ceylon","September 10, 1967, Pyrford, Woking, Surrey (a...",nan,Hampshire,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,16715,Brian Lander,England,Brian Richard Lander,"\nJanuary 9, 1942, Bishop Auckland, Co Durham",nan,77 years 73 days,"['Durham,', 'Minor Counties']",nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,15989,Derek Kenderdine,England,Derek Charles Kenderdine,"\nOctober 28, 1897, Chislehurst, Kent","August 28, 1947, Cambridge (aged 49 years 304 ...",nan,Royal Navy,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
main.columns.tolist()

['ID',
 'NAME',
 'COUNTRY',
 'Full name',
 'Born',
 'Died',
 'Current age',
 'Major teams',
 'Education',
 'Height',
 'Nickname',
 'Playing role',
 'Batting style',
 'Bowling style',
 'Other',
 'Relation',
 'In a nutshell',
 'DESCRIPTION',
 'AWARDS',
 'BATTING_Tests_Mat',
 'BATTING_Tests_Inns',
 'BATTING_Tests_NO',
 'BATTING_Tests_Runs',
 'BATTING_Tests_HS',
 'BATTING_Tests_Ave',
 'BATTING_Tests_BF',
 'BATTING_Tests_SR',
 'BATTING_Tests_100',
 'BATTING_Tests_50',
 'BATTING_Tests_4s',
 'BATTING_Tests_6s',
 'BATTING_Tests_Ct',
 'BATTING_Tests_St',
 'BATTING_ODIs_Mat',
 'BATTING_ODIs_Inns',
 'BATTING_ODIs_NO',
 'BATTING_ODIs_Runs',
 'BATTING_ODIs_HS',
 'BATTING_ODIs_Ave',
 'BATTING_ODIs_BF',
 'BATTING_ODIs_SR',
 'BATTING_ODIs_100',
 'BATTING_ODIs_50',
 'BATTING_ODIs_4s',
 'BATTING_ODIs_6s',
 'BATTING_ODIs_Ct',
 'BATTING_ODIs_St',
 'BATTING_T20Is_Mat',
 'BATTING_T20Is_Inns',
 'BATTING_T20Is_NO',
 'BATTING_T20Is_Runs',
 'BATTING_T20Is_HS',
 'BATTING_T20Is_Ave',
 'BATTING_T20Is_BF',
 'BATTIN

In [ ]:
main['In a nutshell'].isnull().sum()

0

## Dropping those columns which have a high propotion of NULL Values

#### Pyspark

In [ ]:
null_instances=main.isnull().sum()

In [ ]:
null_instances_spark=main_spark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in main_spark.columns])

In [ ]:
null_instances_spark.show(1)

+---+----+-------+---------+----+-----+-----------+-----------+---------+------+--------+------------+-------------+-------------+-----+--------+-------------+-----------+------+-----------------+------------------+----------------+------------------+----------------+-----------------+----------------+----------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+-----------------+---------------+-----------------+---------------+----------------+---------------+---------------+----------------+---------------+---------------+---------------+---------------+---------------+-----------------+------------------+----------------+------------------+----------------+-----------------+----------------+----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------------+------------------------+----------------------+------------------------+----

In [ ]:
null_instances_spark_rdd=null_instances_spark.rdd

In [ ]:
null_instances_spark_rdd.collect()

[Row(ID=0, NAME=2, COUNTRY=0, Full name=2, Born=0, Died=73208, Current age=41892, Major teams=6271, Education=84010, Height=88536, Nickname=90308, Playing role=86590, Batting style=33056, Bowling style=46223, Other=83664, Relation=85608, In a nutshell=90283, DESCRIPTION=0, AWARDS=0, BATTING_Tests_Mat=86656, BATTING_Tests_Inns=86656, BATTING_Tests_NO=86656, BATTING_Tests_Runs=86656, BATTING_Tests_HS=86656, BATTING_Tests_Ave=86656, BATTING_Tests_BF=88625, BATTING_Tests_SR=88625, BATTING_Tests_100=86656, BATTING_Tests_50=86656, BATTING_Tests_4s=88299, BATTING_Tests_6s=87110, BATTING_Tests_Ct=86656, BATTING_Tests_St=86656, BATTING_ODIs_Mat=86652, BATTING_ODIs_Inns=86652, BATTING_ODIs_NO=86652, BATTING_ODIs_Runs=86652, BATTING_ODIs_HS=86652, BATTING_ODIs_Ave=86652, BATTING_ODIs_BF=87240, BATTING_ODIs_SR=87240, BATTING_ODIs_100=86652, BATTING_ODIs_50=86652, BATTING_ODIs_4s=87421, BATTING_ODIs_6s=87400, BATTING_ODIs_Ct=86652, BATTING_ODIs_St=86652, BATTING_T20Is_Mat=88157, BATTING_T20Is_Inns=

In [ ]:
def col_func(x):
    list_=[]
    for col in list_columns:
        propotion_null=x[col]/count_rows
        if propotion_null<0.9:
            list_.append(col)
    return list_
        

In [ ]:
list_columns=main_spark.columns
count_rows=main_spark.count()

In [ ]:
count_rows

90308

In [ ]:
columns_needed=null_instances_spark_rdd.map(col_func).collect()

In [ ]:
null_instances=null_instances.map(lambda x:x/main.shape[0])

In [ ]:
null_instances[null_instances>0.90].index

Index(['Nickname', 'BATTING_Tests_Mat', 'BATTING_ODIs_Mat',
       'BATTING_T20Is_Mat', 'BATTING_T20s_Mat', 'BOWLING_Tests_Mat',
       'BOWLING_ODIs_Mat', 'BOWLING_T20Is_Mat', 'BOWLING_T20s_Mat'],
      dtype='object')

In [ ]:
main_spark=main_spark.drop('Education', 'Height', 'Nickname', 'Playing role', 'Other', 'Relation',
       'In a nutshell')

#### Pandas

In [ ]:
main.drop(['Education', 'Height', 'Nickname', 'Playing role', 'Other', 'Relation',
       'In a nutshell'],1,inplace=True)

In [ ]:
main["Born"]=main["Born"].str.replace("\n","")

## Forming the Date Birth Column which have Year+Date the player was born

#### Pyspark

In [ ]:
split_col = split(main_spark['Born'], ',')

In [ ]:
main_spark = main_spark.withColumn('NAME1', split_col.getItem(0))
main_spark = main_spark.withColumn('NAME2', split_col.getItem(1))

In [ ]:
main_spark.select("NAME1","NAME2").show()

+---------------+--------+
|          NAME1|   NAME2|
+---------------+--------+
|      \nJune 26|    1898|
|     \nApril 25|    1734|
|     \nApril 14|    1882|
|    \nJanuary 9|    1942|
|   \nOctober 28|    1897|
|     \nMarch 24|    1979|
|      \nJuly 17|   1999 |
|   \nDecember 6|    1993|
|  \nFebruary 26|    1929|
|   \nJanuary 21|   2003 |
|\ndate unknown |    null|
|\ndate unknown |    null|
|     \nApril 23|    1986|
|    \nJanuary 8|    1999|
|\ndate unknown |    null|
|\ndate unknown |    null|
|     \nMarch 27|    1991|
|    \nOctober 1|    1987|
|         \n1947| Mombasa|
|        \n1975 |    null|
+---------------+--------+
only showing top 20 rows



In [ ]:
main_spark=main_spark.na.fill('',["NAME2"])

In [ ]:
def fill_Date_birth_spark(x):
    name_2=x["NAME2"]
    name_1=x['NAME1']
    
    name_2=name_2.strip()
    name_1=name_1.strip()
    
    if not name_2.isnumeric():
        return name_1
    else:
        return name_1+"-"+name_2
   

In [ ]:
main_spark_rdd=main_spark.rdd

In [ ]:
main_spark_rdd_names_join=main_spark_rdd.map(fill_Date_birth_spark)

In [ ]:
list_rdd_born=main_spark_rdd_names_join.collect()

In [ ]:
main_spark = main_spark.repartition(1).withColumn(
    "Date_Birth", 
    udf(lambda id: list_rdd_born[id])(monotonically_increasing_id()))

In [ ]:
main_spark=main_spark.drop("NAME1","NAME2")

In [ ]:
main_spark.select("Date_Birth").show()

+----------------+
|      Date_Birth|
+----------------+
|    June 26-1898|
|   April 25-1734|
|   April 14-1882|
|  January 9-1942|
| October 28-1897|
|   March 24-1979|
|    July 17-1999|
| December 6-1993|
|February 26-1929|
| January 21-2003|
|    date unknown|
|    date unknown|
|   April 23-1986|
|  January 8-1999|
|    date unknown|
|    date unknown|
|   March 27-1991|
|  October 1-1987|
|            1947|
|            1975|
+----------------+
only showing top 20 rows



#### Pandas

In [ ]:
date_splitted=main["Born"].str.split(",",expand=True)
date_splitted.iloc[:,1]=date_splitted.iloc[:,1].fillna('')

In [ ]:
def fill_date_birth(x):
    x[1]=x[1].strip()
    if not x[1].isnumeric():
        return x[0]
    else:
        return x[0]+"-"+x[1]

main["Date_Birth"]=date_splitted.apply(fill_date_birth,axis=1)

### What to do with players whose date is not known

#### PySpark

In [ ]:
main_spark = main_spark.withColumn('Date_Birth', trim(main_spark.Date_Birth))

In [ ]:
mode_date_birth_spark=main_spark.filter(~main_spark.Date_Birth.contains('unknown')).groupby('Date_Birth').count().orderBy('count',ascending=False).first()[0]

In [ ]:
main_spark=main_spark.withColumn('Date_Birth', when(main_spark.Date_Birth.rlike('known|Known|circa|Details|Hong|Cheema|circ|Pakistan|Goa|Kamrup|date unknown'),"January 1-"+mode_date_birth_spark).otherwise(
    main_spark['Date_Birth']
))


In [ ]:
main_spark=main_spark.withColumn('Date_Birth', regexp_replace('Date_Birth', ' ', '/')) 
main_spark=main_spark.withColumn('Date_Birth', regexp_replace('Date_Birth', '-', '/')) 
main_spark.select('Date_Birth').show()

+----------------+
|      Date_Birth|
+----------------+
|    June/26/1898|
|   April/25/1734|
|   April/14/1882|
|  January/9/1942|
| October/28/1897|
|   March/24/1979|
|    July/17/1999|
| December/6/1993|
|February/26/1929|
| January/21/2003|
|  January/1/1994|
|  January/1/1994|
|   April/23/1986|
|  January/8/1999|
|  January/1/1994|
|  January/1/1994|
|   March/27/1991|
|  October/1/1987|
|            1947|
|            1975|
+----------------+
only showing top 20 rows



In [ ]:
def date_birth_func_spark(x):
    month_to_numeric={
        "January":"01",
        "February":"02",
        "March":"03",
        "April":"04",
        "May":"05",
        "June":"06",
        "July":"07",
        "August":"08",
        "September":"09",
        "October":"10",
        "November":"11",
        "December":"12",
        "Q1":"1" 
    }
    
    date_instance=x.split("/")
    if '' in date_instance:
        date_instance[date_instance.index('')]='1994'
    if '0000' in date_instance:
        date_instance[date_instance.index('0000')]='1994'
    if len(date_instance)==1:
        return "01/01/"+date_instance[0]
    else:
        if date_instance[0] in month_to_numeric.keys():
            date_instance[0]=month_to_numeric[date_instance[0]]
        return "/".join(date_instance)

In [ ]:
udf_function = udf(lambda x: date_birth_func_spark(x),StringType())
main_spark=main_spark.withColumn('Date_Birth',udf_function(col('Date_Birth')))

In [ ]:
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

In [ ]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
main_spark.withColumn('Date_Birth', from_unixtime(unix_timestamp('Date_Birth', 'MM/dd/yyy'))).select('Date_Birth').show()

+-------------------+
|         Date_Birth|
+-------------------+
|1898-06-26 00:00:00|
|1734-04-25 00:00:00|
|1882-04-14 00:00:00|
|1942-01-09 00:00:00|
|1897-10-28 00:00:00|
|1979-03-24 00:00:00|
|1999-07-17 00:00:00|
|1993-12-06 00:00:00|
|1929-02-26 00:00:00|
|2003-01-21 00:00:00|
|1994-01-01 00:00:00|
|1994-01-01 00:00:00|
|1986-04-23 00:00:00|
|1999-01-08 00:00:00|
|1994-01-01 00:00:00|
|1994-01-01 00:00:00|
|1991-03-27 00:00:00|
|1987-10-01 00:00:00|
|1947-01-01 00:00:00|
|1975-01-01 00:00:00|
+-------------------+
only showing top 20 rows



#### Pandas

In [ ]:
main["Date_Birth"]=main["Date_Birth"].str.strip()

In [ ]:
mode_date_birth=main.loc[~(main["Date_Birth"].str.contains('unknown')),'Date_Birth'].mode().values.tolist()[0]

In [ ]:
main.loc[(main["Date_Birth"].str.contains('known|Known|circa|Details|Hong|Cheema|circ|Pakistan|Goa|Kamrup|Mansa|Matale|Badulla|Balapitiya|Kiribathgoda|Colombo|Westminster|Polonnaruwa')),'Date_Birth']="January 1-"+mode_date_birth

In [ ]:
main["Date_Birth"]=main["Date_Birth"].str.replace(" ","/")
main["Date_Birth"]=main["Date_Birth"].str.replace("-","/")

In [ ]:
final_list=[]
def date_birth_func(x):
    month_to_numeric={
        "January":"01",
        "February":"02",
        "March":"03",
        "April":"04",
        "May":"05",
        "June":"06",
        "July":"07",
        "August":"08",
        "September":"09",
        "October":"10",
        "November":"11",
        "December":"12",
        "Q1":"1" 
    }
    if x=='':
        return datetime.strptime("01/01/"+mode_date_birth, '%m/%d/%Y')
    date_instance=x.split("/")
    try:
        if '' in date_instance:
            date_instance[date_instance.index('')]=mode_date_birth
            return datetime.strptime("/".join(date_instance), '%m/%d/%Y')
        if '0000' in date_instance:
            date_instance[date_instance.index('0000')]=mode_date_birth
        if len(date_instance)==1:
            return datetime.strptime("01/01/"+date_instance[0], '%m/%d/%Y')
        elif len(date_instance)==2:
            date_instance[0]=month_to_numeric[date_instance[0]]
            return datetime.strptime(date_instance[0]+"/01/"+date_instance[1], '%m/%d/%Y')
        else:
            if date_instance[0] in month_to_numeric.keys():
                date_instance[0]=month_to_numeric[date_instance[0]]
            return datetime.strptime("/".join(date_instance), '%m/%d/%Y')
    except:
        final_list.append(x)
   
        
    
    
   
    
    

In [ ]:
main["Date_Birth"].map(date_birth_func)

0       1898-06-26
1       1734-04-25
2       1882-04-14
3       1942-01-09
4       1897-10-28
           ...    
90303   1933-12-17
90304   1861-12-03
90305   1825-07-02
90306   1857-11-17
90307   1994-01-01
Name: Date_Birth, Length: 90308, dtype: datetime64[ns]

## Forming the Date Death Column which have Year+Date the player was born

#### Pyspark

In [ ]:
split_col = split(main_spark['Died'], ',')

In [ ]:
main_spark = main_spark.withColumn('NAME1', split_col.getItem(0))
main_spark = main_spark.withColumn('NAME2', split_col.getItem(1))

In [ ]:
main_spark=main_spark.fillna('',subset=['NAME2'])

In [ ]:
custom_func_name1_name2=udf(lambda x,y:x+"-"+y)

In [ ]:
main_spark=main_spark.withColumn('Died',custom_func_name1_name2(col('NAME1'),col('NAME2')))

In [ ]:
main_spark=main_spark.drop('NAME1','NAME2')

In [ ]:
mode_date_died_spark=main_spark.filter(~main_spark.Died.rlike('nan|unknown|Presumed Dead|Known|Unknown')).groupby('Died').count().orderBy('count',ascending=False).first()[0]

In [ ]:
mode_date_died_spark

'September 25- 1915'

In [ ]:
mode_year_spark=mode_date_died_spark.split('- ')[-1]

In [ ]:
main_spark=main_spark.withColumn('Died', when(main_spark.Died.rlike('unknown|Presumed Dead|Known|Unknown|nan|0000|aged|Duncton|Leicester|Tasmania'),mode_date_died_spark).otherwise(
    main_spark['Died']
))

In [ ]:
main_spark.select('Died').show()

+------------------+
|              Died|
+------------------+
|    March 12- 1982|
|    April 25- 1797|
|September 10- 1967|
|September 25- 1915|
|   August 28- 1947|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
|September 25- 1915|
+------------------+
only showing top 20 rows



In [ ]:
main_spark=main_spark.withColumn('Died', regexp_replace('Died', '- ', '/')) 
main_spark=main_spark.withColumn('Died', regexp_replace('Died', ' ', '/')) 


####### Implement the dead function for Pyspark

#### Pandas

In [ ]:
main['Died']=main['Died'].map(lambda x:"-".join(x.split(',')[:2]))

In [ ]:
mode_died=main.loc[~(main['Died'].str.contains('nan|unknown|Presumed Dead|Known|Unknown')),'Died'].mode().values.tolist()[0]

In [ ]:
mode_year=mode_died.split('- ')[-1]

In [ ]:
main.loc[(main['Died'].str.contains('unknown|Presumed Dead|Known|Unknown|nan|0000|aged|Duncton|Leicester|Tasmania')),'Died']=mode_died

In [ ]:
main['Died']=main['Died'].str.replace('- ','/').str.replace(' ','/')

In [ ]:
def date_death_func(x):
    month_to_numeric={
        "January":"01",
        "February":"02",
        "March":"03",
        "April":"04",
        "May":"05",
        "June":"06",
        "July":"07",
        "August":"08",
        "September":"09",
        "October":"10",
        "November":"11",
        "December":"12",
        "Q1":"1" ,
        'Q2':'4'
    }  
   

    date_list=x.split('/')[:3]
    all_numeric=True
    index_to_pop=[]
    for index,instance in enumerate(date_list):
        if not instance.isnumeric() and instance not in month_to_numeric.keys():
            all_numeric=False
        else:    
            index_to_pop.append(index)
    if all_numeric:
        date_list[0]=month_to_numeric[date_list[0]]
        return datetime.strptime('/'.join(date_list), '%m/%d/%Y')
        
    else:

        new_date_list=[]
        for index in index_to_pop:
            new_date_list.append(date_list[index])
        if len(new_date_list)==1:
            return datetime.strptime('01/'+'01/'+new_date_list[0], '%m/%d/%Y')
        elif len(new_date_list)==2 and new_date_list[0] in month_to_numeric.keys():
            new_date_list[0]=month_to_numeric[new_date_list[0]]
            return datetime.strptime(new_date_list[0]+'/01/'+new_date_list[1], '%m/%d/%Y')
        else:
            return datetime.strptime('01/'+'01/'+mode_year, '%m/%d/%Y')
                
    
        
            
    
      

In [ ]:
main['Died'].map(date_death_func)

0       1982-03-12
1       1797-04-25
2       1967-09-10
3       1915-09-25
4       1947-08-28
           ...    
90303   1915-09-25
90304   1953-11-02
90305   1907-01-29
90306   1938-04-01
90307   1915-09-25
Name: Died, Length: 90308, dtype: datetime64[ns]